[How to Read Outlook Emails by Python](https://www.codementor.io/aliacetrefli/how-to-read-outlook-emails-by-python-jkp2ksk95)

In [ ]:
import win32com.client
import win32com
import os
import sys
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

## Create outlook mapi and outlook accounts objects

In [ ]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
accounts = win32com.client.Dispatch("Outlook.Application").Session.Accounts;

## To quit the com objects

In [ ]:
outlook.Application.quit()  

In [ ]:
accounts.Application.quit()

## Parse all emails from all folders from all accounts for sender email addressses

In [ ]:
def Apple(folder):
    messages = folder.Items
    a=len(messages)
    if a>0:
        for message2 in messages:
            try:
                sender = message2.SenderEmailAddress
                if sender != "":
                    print(sender, file=f)
            except:
                print("Error")
                print(account.DeliveryStore.DisplayName)
                pass
            try:
                message2.Save
                message2.Close(0)
            except:
                 pass

In [ ]:
with open('testfile.txt', 'w') as f:
    for account in accounts:
        global inbox
        inbox = outlook.Folders(account.DeliveryStore.DisplayName)
        print("****Account Name**********************************",file=f)
        print(account.DisplayName,file=f)
        print(account.DisplayName)
        print("***************************************************",file=f)
        folders = inbox.Folders

        for folder in folders:
            print("****Folder Name**********************************", file=f)
            print(folder, file=f)
            print("*************************************************", file=f)
            Apple(folder)
            a = len(folder.folders)

            if a>0 :
                global z
                z = outlook.Folders(account.DeliveryStore.DisplayName).Folders(folder.name)
                x = z.Folders
                for y in x:
                    Apple(y)
                    print("****Folder Name**********************************", file=f)
                    print("..."+y.name,file=f)
                    print("*************************************************", file=f)
print("Finished Succesfully")

## Read, parse and count only the email addresses from all folders

In [ ]:
df = pd.read_csv('testfile.txt', sep='\n', header=None)

In [ ]:
df.head()

In [ ]:
df = df[df[0].str.contains('@')]  # filters out all non-email address rows

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df = df[0].str.lower()

In [ ]:
df.value_counts()

* [Reading e-mails from Outlook with Python through MAPI](https://stackoverflow.com/questions/5077625/reading-e-mails-from-outlook-with-python-through-mapi)
* [MailItem Interface](https://docs.microsoft.com/en-us/dotnet/api/microsoft.office.interop.outlook.mailitem?redirectedfrom=MSDN&view=outlook-pia#properties_)
* [OlDefaultFolders Enumeration (Outlook)](https://docs.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders)
* [Items Object (Outlook)](https://docs.microsoft.com/en-us/office/vba/api/outlook.items)
* [ContactItem Object (Outlook)](https://docs.microsoft.com/en-us/office/vba/api/outlook.contactitem)

## Figuring out how to retrieve information from the message

In [ ]:
inbox = outlook.GetDefaultFolder(6)  # 6 is the inbox

In [ ]:
inbox.Folders

In [ ]:
messages = inbox.Items

In [ ]:
messages

In [ ]:
messages(2).Display()  # opens the email

In [ ]:
message = messages.GetFirst()

In [ ]:
print(message.Subject)
print(message.To)
print(message.ReceivedTime)
print(message.Sender)
print(message.SenderEmailAddress)
print(message.SentOn)

In [ ]:
print(message.body)

In [ ]:
type(message.body)

In [ ]:
with open('data.txt', 'w') as f:
    f.write(message.body)

In [ ]:
import re

In [ ]:
word = re.sub(r'\n+', '\n', message.body).strip()

In [ ]:
word

In [ ]:
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items
y = messages.GetLast()

with open('messages.txt', 'w') as f:
    f.write(f'{y.Subject}\n')
    f.write(f'{y.To}\n')
    f.write(f'{str(y.ReceivedTime)}\n')
    f.write(f'{str(y.Sender)}\n')
    f.write(f'{y.SenderEmailAddress}\n')
    f.write(f'{str(y.SentOn)}\n')
    f.write(y.body)

    i = 0
    for x in range(len(messages)-1):
        i+=1
        y = messages.GetPrevious()

        try: 
            f.write('*' * 100)
            f.write('*' * 100)
            f.write(f'{y.Subject}\n')
            f.write(f'{y.To}\n')
            f.write(f'{str(y.ReceivedTime)}\n')
            f.write(f'{str(y.Sender)}\n')
            f.write(f'{y.SenderEmailAddress}\n')
            f.write(f'{str(y.SentOn)}\n')
#             f.write(y.body)


        except AttributeError as error:
            print(error)

        if i > 10:
            break

## Parse Inbox and create list of dicts

In [ ]:
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items

list_dict_messages = list()
messages_length = len(messages)

for x in range(messages_length):
    if x == 0:
        print(f'Messages to parse: {messages_length}')
        y = messages.GetLast()
    else:
        y = messages.GetPrevious()
        
    
    try:
        sent_date = y.SentOn
    except AttributeError:
        sent_date = ''
    try:
        received_date = y.ReceivedTime
    except AttributeError:
        received_date = ''
    try:
        from_name = y.Sender
    except AttributeError:
        from_name = ''
    try:
        from_address = y.SenderEmailAddress
    except AttributeError:
        from_address = ''
    try:
        recipient = y.To
    except AttributeError:
        recipient = ''
    try:
        subject = y.Subject
    except AttributeError:
        subject = ''
    try:
        body = y.body
    except AttributeError:
        body = ''
        
    email_info = {'sent_date': sent_date, 'received_date': received_date, 'from_name': from_name,
                  'from_address': from_address, 'recipient': recipient, 'subject': subject, 'body': body}
    
    list_dict_messages.append(email_info)
    if x % 100 == 0:
        print(f'Messages parsed: {x+1}')

outlook.Application.Quit()
accounts.Application.Quit()

## Create DataFrame

In [ ]:
df = pd.DataFrame(list_dict_messages, dtype='str')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.from_name.fillna(df.from_address, inplace=True)

In [ ]:
df.info()

## Save to csv

In [ ]:
df.to_csv('all_emails.csv', index=False)

## Read from csv

In [ ]:
df = pd.read_csv('all_emails.csv', parse_dates=['received_date', 'sent_date'])

## Parse out all links from email body

In [ ]:
df['links_in_body'] = df.body.apply(lambda x: list(set(re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', str(x)))))

In [ ]:
df.from_name.fillna(df.from_address, inplace=True)

In [ ]:
df.fillna('', inplace=True)

In [ ]:
df.sent_date = pd.to_datetime(df.sent_date, errors='coerce')

In [ ]:
df.info()

In [ ]:
df.head()

## Create LinkedIn DataFrame

In [ ]:
linkedin = df[df.from_address.str.contains('linkedin')]  # inside the braces is a boolean filter

In [ ]:
linkedin.info()

In [ ]:
for i, x in enumerate(linkedin.body.tail(1)):
    print(f'{"*"*150}\nMessage: {i}\n{x}')

## Parse all links into a list (remove http & https)

In [ ]:
links_list = list()
for y in df.links_in_body:
    if y:
        links_list.extend(y)

In [ ]:
links_list[:5]

In [ ]:
links_list2 = list()
for x in links_list:
    y = x.split('/')
    links_list2.append(y[-1])

In [ ]:
links_list2[:5]

## Unique counts of links

In [ ]:
links_counts = Counter(links_list2)

In [ ]:
links_counts.most_common(10)

In [ ]:
links_counts.most_common()[-10:]

In [ ]:
df['date'] = df.sent_date.dt.normalize()

In [ ]:
df.info()

In [ ]:
test = df[['date', 'from_name']].groupby(['date']).count()

In [ ]:
test.head()

## Plots of email counts

In [ ]:
test.plot(kind='bar', legend=False, figsize=(15,9))
plt.ylabel('email counts')
plt.show()

In [ ]:
year_list = ['2019', '2020']

for x in year_list:
    test.loc[x].plot(legend=False, figsize=(7,4), subplots=True)
    plt.ylabel('email counts')
    plt.xlabel(f'Year: {x}')
plt.show()

## This section has to be run from a conda environment with pyviz

In [ ]:
import time
# import param

import numpy as np
from bokeh.sampledata import stocks
import holoviews as hv
from holoviews import opts

from holoviews.operation.timeseries import rolling, rolling_outlier_std
from holoviews.operation.datashader import datashade, dynspread

In [ ]:
hv.extension('bokeh')

opts.defaults(opts.Curve(width=600, framewise=True))

In [ ]:
def load_symbol(years, **kwargs):
    df3 = test.loc[years]
    return hv.Curve(df3, ('date', 'Date'), ('from_name', 'Number of emails'))

year_list = ['2009', '2010', '2011', '2012', '2013', '2014']
dmap = hv.DynamicMap(load_symbol, kdims='Year').redim.values(Year=year_list)

In [ ]:
dmap